In [25]:
# FUGU code module
import networkx as nx




In [ ]:
class Circuit:
    """Prototype circuit class for Fugu - where we will build output Fugu networks"""
    
    
    
    def __init__(self):
        circuit_abstract=nx.DiGraph()
    
    def add_brick(self, brick_func, *arg):
        # Function to add brick to circuit
        #
        # brick_func will call procedure to generate the subcircuit
        # * arg will be arguments to pass into brick_func, if appropriate
        n_nodes=self.circuit_abstract.number_of_nodes()
        self.circuit_abstract.add_node(n_nodes+1, )
        self.circuit_abstract.add_edge()
        